In [2]:
import ray
ray.init()

2022-11-08 15:24:24,606	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.9.5
Ray version:,2.0.1


# Basic Example

## Load Data

Basic data that can be understood as two different input columns (linear and random values) and three different output columns (sin of linear x, 1 + sin of linear x and random values).

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("example_data.csv", dtype=float)

In [5]:
data.head()

,lin_x,rand_x,sinx,randomy,sinx_plus1
0,1.0,-0.55,0.099833,0.09,1.099833
1,2.0,-0.44,0.198669,0.83,1.198669
2,3.0,0.94,0.295520,0.12,1.295520
3,4.0,-0.61,0.389418,0.41,1.389418
4,5.0,-0.51,0.479426,-0.70,1.479426


## Get Combinations function

This function can be used to determine the overall number of combinations the predictability routine analyses given the number of data columns, fitting type etc. This allows to estimate the overall runtime of the predictability routine.

In [6]:
from src.ASD_predictability_utils.utils import get_column_combinations

The function returns a list of combination tuples, where the first inputs-many elements correspond to the inputs and the remaining ouputs-many to the targets. The argument targets can be used to define columns that should exclusively be regarded as targets.

In [7]:
# applied to a numerical example:
get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=3,
                                  outputs=1,
                                  targets=[5, 6]
                                  )

[(1, 2, 3, 5),
 (1, 2, 3, 6),
 (1, 2, 4, 5),
 (1, 2, 4, 6),
 (1, 3, 4, 5),
 (1, 3, 4, 6),
 (2, 3, 4, 5),
 (2, 3, 4, 6)]

In [8]:
# the argument "amount_only" can be used to output the amout of combinations only
get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=3,
                                  outputs=1,
                                  targets=[5, 6],
                                  amount_only=True
                                  )

8

In [9]:
# applied to the data above
print(get_column_combinations(all_cols=data.columns, inputs=1, outputs=1, targets=["sinx", "randomy", "sinx_plus1"]))

[('lin_x', 'sinx'), ('lin_x', 'randomy'), ('lin_x', 'sinx_plus1'), ('rand_x', 'sinx'), ('rand_x', 'randomy'), ('rand_x', 'sinx_plus1')]


In [10]:
print(get_column_combinations(all_cols=data.columns, inputs=1, outputs=1, targets=["sinx", "randomy", "sinx_plus1"],
                                  amount_only=True))

6


## Predictability function

Running the predictability function over all possible 1+1 combinations where the respective target is either sin, sin+1 or random and the input is linear x or random values.

The purpose of having sin + 1 is to have a tuple that exclusively contains positive values so fitting a power law can be applied.

In [11]:
from bin.main import predictability

In [12]:
metrics_dict, datas_dict = predictability(data=data,
                                          input_cols=1,
                                          output_cols=1,
                                          col_set=None,
                                          targets=["sinx", "randomy", "sinx_plus1"],
                                          method="kNN",
                                          random_state_split=None
                                          )

(parallel_pred_step_kNN pid=102781) Analysing ('rand_x', 'sinx_plus1') now.
(parallel_pred_step_kNN pid=102781) start kNN routine
(parallel_pred_step_kNN pid=102781) Fitting 3 folds for each of 6 candidates, totalling 18 fits
(parallel_pred_step_kNN pid=102786) Analysing ('lin_x', 'sinx_plus1') now.
(parallel_pred_step_kNN pid=102786) start kNN routine
(parallel_pred_step_kNN pid=102786) Fitting 3 folds for each of 6 candidates, totalling 18 fits
(parallel_pred_step_kNN pid=102787) Analysing ('lin_x', 'sinx') now.
(parallel_pred_step_kNN pid=102787) start kNN routine
(parallel_pred_step_kNN pid=102787) Fitting 3 folds for each of 6 candidates, totalling 18 fits
(parallel_pred_step_kNN pid=102778) Analysing ('lin_x', 'randomy') now.
(parallel_pred_step_kNN pid=102778) start kNN routine
(parallel_pred_step_kNN pid=102778) Fitting 3 folds for each of 6 candidates, totalling 18 fits
(parallel_pred_step_kNN pid=102783) Analysing ('rand_x', 'sinx') now.
(parallel_pred_step_kNN pid=102783) st

##### Note that starting the Ray instance accounts for most of the runtime. Start the cell above again and runtime will be <1s.

In [13]:
pd.DataFrame.from_dict(metrics_dict).transpose()

kNN r2  linear r2  pow. law r2   mean r2  kNN RMSE  \
lin_x  sinx        0.927973  -0.103760          NaN -0.031741  0.180688   
       randomy     0.076147  -0.005914          NaN -0.005303  0.600267   
       sinx_plus1  0.986451  -0.205439    -0.959307 -0.077711  0.077002   
rand_x sinx        0.001501   0.004933          NaN -0.013052  0.632834   
       randomy    -0.326034  -0.220654          NaN -0.211552  0.690875   
       sinx_plus1 -0.121684  -0.139449          NaN -0.169557  0.704903   

                   linear RMSE  pow. law RMSE  mean RMSE  kNN RMSE/std  \
lin_x  sinx           0.707328            NaN   0.683863      0.268378   
       randomy        0.626360            NaN   0.626169      0.961173   
       sinx_plus1     0.726310       0.925978   0.686753      0.116401   
rand_x sinx           0.631745            NaN   0.637429      0.999249   
       randomy        0.662854            NaN   0.660378      1.151536   
       sinx_plus1     0.710463            NaN   0.719788      1.059096   

                   linear RMSE/std  ...  pow. law MAPE   mean MAPE    kNN rae  \
lin_x  sinx               1.050600  ...            NaN    1.025132  36.373312   
       randomy            1.002953  ...            NaN    0.997500  29.864493   
       sinx_plus1         1.097925  ...     304.297736  504.329381  38.221088   
rand_x sinx               0.997530  ...            NaN    0.977183  32.306825   
       randomy            1.104832  ...            NaN    1.450535  35.179767   
       sinx_plus1         1.067450  ...            NaN  557.091293  31.701710   

                   linear rae  pow. law rae   mean rae  kNN dcor  linear dcor  \
lin_x  sinx         30.354678           NaN  30.225325  0.969348     0.472007   
       randomy      29.657798           NaN  29.659535  0.459352     0.242814   
       sinx_plus1   35.267212     45.202231  33.264498  0.990308     0.435462   
rand_x sinx         31.637759           NaN  30.723015  0.274593     0.319025   
       randomy      34.545465           NaN  34.264513  0.234720     0.263941   
       sinx_plus1   31.119089           NaN  30.952806  0.354361     0.358192   

                   pow. law dcor  mean dcor  
lin_x  sinx                  NaN   0.000214  
       randomy               NaN   0.000000  
       sinx_plus1       0.426597   0.000000  
rand_x sinx                  NaN   0.000193  
       randomy               NaN   0.000000  
       sinx_plus1            NaN   0.000000  

[6 rows x 24 columns]

## Tuple Selection function

This function can be used to limit the number of tuples that is further analysed in more detail.

In [14]:
from bin.main import tuple_selection

In [15]:
selected_tuples = tuple_selection(metrics_dict, n_best=2)
selected_tuples

[('lin_x', 'sinx_plus1'), ('lin_x', 'sinx')]

## Refine Predictability function

This function is used to further refine the predictability of the previously selected best tuples.

In [16]:
from bin.main import refine_predictability

In [17]:
refined_metrics_dict, refined_datas_dict = refine_predictability(best_tuples=selected_tuples, data_dict=datas_dict, time_left_for_this_task=120,
                          per_run_time_limit=30, n_jobs=-1)

(parallel_refinement_step pid=102789) /home/dw/Projekte/IPP/asd/venv/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
(parallel_refinement_step pid=102789) Perhaps you already have a cluster running?
(parallel_refinement_step pid=102789) Hosting the HTTP server on port 44551 instead
(parallel_refinement_step pid=102789)   warnings.warn(


(parallel_refinement_step pid=102786) Train R2 score: 0.9476159378347307
(parallel_refinement_step pid=102786) Test R2 score: 0.9580847241030104


In [18]:
pd.DataFrame.from_dict(refined_metrics_dict).transpose()

r2      RMSE      MAPE        rae      dcor
lin_x sinx_plus1  0.999999  0.000562  1.336834  38.394055  1.000000
      sinx        0.958085  0.137838  0.212106  35.932202  0.989433

In [24]:
for key in refined_datas_dict[list(refined_datas_dict.keys())[0]]["ensemble_models"]["sklearn_regressor"].keys():
    print(f'Weight: {refined_datas_dict[list(refined_datas_dict.keys())[0]]["ensemble_models"]["ensemble_weight"][key]}, \t Regressor: {refined_datas_dict[list(refined_datas_dict.keys())[0]]["ensemble_models"]["sklearn_regressor"][key]}')

Weight: 1.0, 	 Regressor: GaussianProcessRegressor(alpha=5.211916694590534e-14,
                         kernel=RBF(length_scale=1), n_restarts_optimizer=10,
                         normalize_y=True, random_state=1)


In [29]:
refined_datas_dict[list(refined_datas_dict.keys())[0]]["ensemble_models"]

,ensemble_weight,rank,sklearn_regressor
83,1.0,1,GaussianProcessRegressor(alpha=5.2119166945905...
